In [1]:
from progressivis_nb_widgets import ControlPanel, Scatterplot, ModuleGraph, wait_for_click
sc = Scatterplot()
gr = ModuleGraph()

In [2]:
import time
import pandas as pd
import copy
from progressivis.core import Scheduler, Every, JSONEncoderNp, asynchronize
from progressivis.table import PTable
from progressivis.vis import MCScatterPlot
from progressivis.io import CSVLoader
#from progressivis.datasets import get_dataset
from progressivis.table.constant import Constant
import asyncio as aio
import functools as ft

def _quiet(x): pass


def _filter(df):
    pklon = df['pickup_longitude']
    pklat = df['pickup_latitude']
    dolon = df['dropoff_longitude']
    dolat = df['dropoff_latitude']

    return df[(pklon > -74.08) & (pklon < -73.5) & (pklat > 40.55) & (pklat < 41.00) &
                  (dolon > -74.08) & (dolon < -73.5) & (dolat > 40.55) & (dolat < 41.00)]

def feed_widget(wg, val):
    wg.data = JSONEncoderNp.dumps(val)

try:
    s = scheduler
except NameError:
    s = Scheduler()
    
cpanel = ControlPanel(s)

#PREFIX= 'https://storage.googleapis.com/tlc-trip-data/2015/'
#SUFFIX= ''
PREFIX = '../../nyc-taxi/'

SUFFIX = '.bz2'

URLS = [
    PREFIX+'yellow_tripdata_2015-01.csv'+SUFFIX,
    PREFIX+'yellow_tripdata_2015-02.csv'+SUFFIX,
    PREFIX+'yellow_tripdata_2015-03.csv'+SUFFIX,
    PREFIX+'yellow_tripdata_2015-04.csv'+SUFFIX,
    PREFIX+'yellow_tripdata_2015-05.csv'+SUFFIX,
    PREFIX+'yellow_tripdata_2015-06.csv'+SUFFIX,
]

FILENAMES = pd.DataFrame({'filename': URLS})
#import pdb;pdb.set_trace()
CST = Constant(PTable('filenames', data=FILENAMES), scheduler=s)
CSV = CSVLoader(index_col=False, skipinitialspace=True,
                usecols=['pickup_longitude', 'pickup_latitude',
                             'dropoff_longitude', 'dropoff_latitude'],
                filter_=_filter, scheduler=s) # TODO: reimplement filter in read_csv.py

CSV.input.filenames = CST.output[0]
PR = Every(scheduler=s, proc=_quiet)
PR.input.df = CSV.output[0]

MULTICLASS = MCScatterPlot(scheduler=s, classes=[
    ('pickup', 'pickup_longitude', 'pickup_latitude'),
    ('dropoff', 'dropoff_longitude', 'dropoff_latitude')], approximate=True)
MULTICLASS.create_dependent_modules(CSV, 'result')

async def control_panel(action):
    btn, cb =  cpanel.cb_args(action)
    while True:
        await wait_for_click(btn, cb)
    
aio.create_task(s.start())
coros=sc.link_module(MULTICLASS)+[control_panel("resume"), control_panel("stop"), control_panel("step")]
for coro in coros:
    aio.create_task(coro)

# Scheduler added module(s): ['constant_1', 'csv_loader_1', 'every_1', 'histogram_index_1', 'histogram_index_2', 'histogram_index_3', 'histogram_index_4', 'max_1', 'max_2', 'max_3', 'max_4', 'mc_histogram2_d_1', 'mc_histogram2_d_2', 'mc_scatter_plot_1', 'merge_dict_1', 'merge_dict_2', 'merge_dict_3', 'merge_dict_4', 'min_1', 'min_2', 'min_3', 'min_4', 'range_query2d_1', 'range_query2d_2', 'sample_1', 'sample_2', 'variable_1', 'variable_2', 'variable_3', 'variable_4', 'virtual_variable_1', 'virtual_variable_2']
*****

In [3]:
import ipywidgets as ipw
tab = ipw.Tab()
tab.children = [sc, gr]
tab.set_title(0, 'Scatterplot')
tab.set_title(1, 'Module graph')
vbox = ipw.VBox([tab, cpanel])
vbox

******************In scheduler, tasks {<Task finished name='Task-12' coro=<Scheduler.shortcut_manager() done, defined at /home/fekete/src/progressivis/progressivis/core/scheduler.py:126> result=None>, <Task finished name='Task-11' coro=<Scheduler._run_loop() done, defined at /home/fekete/src/progressivis/progressivis/core/scheduler.py:364> result=None>} done
Leaving run()
In scheduler, tasks {<Task finished name='Task-16' coro=<Scheduler.shortcut_manager() done, defined at /home/fekete/src/progressivis/progressivis/core/scheduler.py:126> result=None>, <Task finished name='Task-15' coro=<Scheduler._run_loop() done, defined at /home/fekete/src/progressivis/progressivis/core/scheduler.py:364> result=None>} done
Leaving run()
*In scheduler, tasks {<Task finished name='Task-20' coro=<Scheduler.shortcut_manager() done, defined at /home/fekete/src/progressivis/progressivis/core/scheduler.py:126> result=None>, <Task finished name='Task-19' coro=<Scheduler._run_loop() done, defined at /home/fek